<a href="https://colab.research.google.com/github/fenwickslab/fenwicks/blob/master/tutorials/tutorial3_kaggle_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -qq -U cufflinks

In [0]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import functools

In [3]:
if tf.io.gfile.exists('./fenwicks'):
  tf.io.gfile.rmtree('./fenwicks')
!git clone -q https://github.com/fenwickslab/fenwicks.git

import fenwicks as fw

In [4]:
BATCH_SIZE = 512 #@param ["512", "256", "128"] {type:"raw"}
MOMENTUM = 0.9 #@param ["0.9", "0.95", "0.975"] {type:"raw"}
WEIGHT_DECAY = 0.000125 #@param ["0.000125", "0.00025", "0.0005"] {type:"raw"}
LEARNING_RATE = 0.4 #@param ["0.4", "0.2", "0.1"] {type:"raw"}
EPOCHS = 24 #@param {type:"slider", min:0, max:100, step:1}
WARMUP = 5 #@param {type:"slider", min:0, max:24, step:1}
ROOT_DIR = 'gs://gs_colab'
PROJECT = 'tutorial3'
VALID_BATCH_SIZE = 10000
PRED_BATCH_SIZE = 10000

In [5]:
fw.colab_utils.setup_gcs()

In [6]:
fw.colab_utils.setup_kaggle_from_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./gdrive


In [7]:
!kaggle $fw.datasets.URLs.KAGGLE_CIFAR10

  0% 0.00/3.04M [00:00<?, ?B/s]
100% 3.04M/3.04M [00:00<00:00, 48.3MB/s]
100% 609M/610M [00:11<00:00, 85.8MB/s]
100% 610M/610M [00:11<00:00, 57.7MB/s]
100% 105M/105M [00:03<00:00, 18.4MB/s] 

  0% 0.00/575k [00:00<?, ?B/s]
100% 575k/575k [00:00<00:00, 79.1MB/s]


In [8]:
!apt -qq install libarchive-dev
!apt -qq autoremove
!pip install -qq libarchive

data_dir_local = './data'
fw.io.unzip(['./train.7z', './test.7z'], data_dir_local)

The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  libarchive-dev
0 upgraded, 1 newly installed, 0 to remove and 7 not upgraded.
Need to get 470 kB of archives.
After this operation, 1,621 kB of additional disk space will be used.
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 130824 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.2.2-3.1ubuntu0.3_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.2.2-3.1ubuntu0.3) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Setting up libarchive-dev:amd64 (3.2.2-3.1ubuntu0.3) ...
The following packages will be REMOVED:
  libnvidia-common-410
0 upgraded, 0 newly installed, 1 to remove and 7 not upgraded.
After this operation, 33.8 kB disk space will be freed.
(Reading database ... 130880 files and directories currently installed.)
Removing


INFO:tensorflow:Decompressing: /content/test.7z


In [10]:
data_dir_local_train = os.path.join(data_dir_local, 'train')
data_dir_local_test = os.path.join(data_dir_local, 'test')
fw.image.check_rgb(data_dir_local_train, file_ext='png')
fw.image.check_rgb(data_dir_local_test, file_ext='png')

In [11]:
data_dir, work_dir = fw.io.get_project_dirs(ROOT_DIR, PROJECT)

In [12]:
local_train_fn = os.path.join(data_dir_local, "train.tfrec")

paths_train, y_train, labels = fw.data.data_dir_label_csv_tfrecord(
  data_dir=data_dir_local_train, csv_fn='./trainLabels.csv', output_fn=local_train_fn, file_ext='png')

In [13]:
n_train, n_classes = len(paths_train), len(labels)
n_train, n_classes

(50000, 10)

In [15]:
fw.plt.plot_counts_pie(y_train, labels, w=450)

In [16]:
fw.anim.show_image_files(paths_train, h_inch=1, w_inch=1)

In [18]:
X_train_mean, X_train_std, img_size, _ = fw.image.compute_mean_std(local_train_fn, n_train, batch_size=100)

Instructions for updating:
Colocations handled automatically by placer.


In [19]:
train_fn = os.path.join(data_dir, "train.tfrec")
fw.io.upload_to_gcs(local_train_fn, train_fn)

In [20]:
test_fn = os.path.join(data_dir, "test.tfrec")

path_test = fw.data.data_dir_no_label_tfrecord(data_dir=data_dir_local_test, output_fn=test_fn, file_ext='png')

n_test = len(path_test)
n_test

300000

In [21]:
train_tfms = [
  fw.transform.tfm_standard_scaler(X_train_mean, X_train_std),
  fw.transform.tfm_pad_crop(4),
  fw.transform.tfm_random_flip(),
  fw.transform.tfm_cutout(8, 8),
  fw.transform.tfm_set_shape(img_size, img_size),
]

test_tfms = [
  fw.transform.tfm_standard_scaler(X_train_mean, X_train_std),
  fw.transform.tfm_set_shape(img_size, img_size),
]

parser_train = fw.data.get_tfexample_image_parser(train_tfms)
parser_test = fw.data.get_tfexample_image_parser(test_tfms, has_label=False)

In [22]:
train_input_func = lambda params: fw.data.tfrecord_ds(train_fn, parser_train, batch_size=params['batch_size'], training=True)
test_input_func = lambda params: fw.data.tfrecord_ds(test_fn, parser_test, batch_size=params['batch_size'], training=False)

In [23]:
def build_nn(c=64, weight=0.125):
  model = fw.Sequential()
  model.add(fw.layers.ConvBN(c, **fw.layers.PYTORCH_PARAMS))
  model.add(fw.layers.ConvResBlk(c*2, res_convs=2, **fw.layers.PYTORCH_PARAMS))
  model.add(fw.layers.ConvBlk(c*4, **fw.layers.PYTORCH_PARAMS))
  model.add(fw.layers.ConvResBlk(c*8, res_convs=2, **fw.layers.PYTORCH_PARAMS))
  model.add(tf.keras.layers.GlobalMaxPool2D())
  model.add(fw.layers.Classifier(n_classes, kernel_initializer=fw.layers.init_pytorch, weight=weight))
  return model

In [24]:
steps_per_epoch = n_train // BATCH_SIZE
total_steps = steps_per_epoch * EPOCHS
warmup_steps = steps_per_epoch * WARMUP

In [25]:
lr_decay = fw.train.linear_decay()
lr_func = fw.train.one_cycle_lr(LEARNING_RATE/BATCH_SIZE, total_steps, warmup_steps, lr_decay)
fw.plt.plot_lr_func(lr_func, total_steps)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [26]:
opt_func = fw.train.sgd_optimizer(lr_func, mom=MOMENTUM, wd=WEIGHT_DECAY*BATCH_SIZE)
model_func = fw.train.get_clf_model_func(build_nn, opt_func, reduction=tf.losses.Reduction.SUM)

In [27]:
est = fw.train.get_tpu_estimator(steps_per_epoch, model_func, work_dir, trn_bs=BATCH_SIZE, val_bs=VALID_BATCH_SIZE, pred_bs=PRED_BATCH_SIZE)
est.train(train_input_func, steps=total_steps)

INFO:tensorflow:Using config: {'_model_dir': 'gs://gs_colab/work/tutorial3/2019-05-22-19:20:08', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.59.171.130:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fee4cb8f080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.59.171.130:8470', '_evaluation_master': 'grpc://10.59.171.130:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=97, num_shards=N

In [28]:
y_preds = []
test_ids = []
for i, pred in enumerate(est.predict(test_input_func)):
  y_preds.append(pred['y_pred'])
  fn = os.path.basename(path_test[i])
  test_ids.append(fn[:-4])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from gs://gs_colab/work/tutorial3/2019-05-22-19:20:08/model.ckpt-2328
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 10 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Deque

In [29]:
d = {'id': test_ids, 'label': [labels[y] for y in y_preds]}
df = pd.DataFrame(data=d)
df

,id,label
0,217159,deer
1,253175,truck
2,168749,airplane
3,260574,ship
4,193598,frog
5,73749,deer
6,73922,dog
7,218719,cat
8,97718,dog
9,43007,airplane


In [30]:
df.to_csv('submission.csv', index=False)

In [31]:
!kaggle competitions submit -c cifar-10 -f submission.csv -m "Tutorial 3"

100% 3.54M/3.54M [00:01<00:00, 1.88MB/s]
Successfully submitted to CIFAR-10 - Object Recognition in Images

In [32]:
!kaggle competitions submissions -c cifar-10

fileName        date                 description  status    publicScore  privateScore  
--------------  -------------------  -----------  --------  -----------  ------------  
submission.csv  2019-05-22 19:21:40  Tutorial 3   complete  0.93960      0.93960       
submission.csv  2019-05-15 06:36:24  Tutorial 3   complete  0.94320      0.94320       
submission.csv  2019-05-15 05:57:41  Tutorial 3   complete  0.09710      0.09710       
submission.csv  2019-05-12 05:46:00  Tutorial 3   complete  0.94200      0.94200       
submission.csv  2019-05-08 15:36:23  Tutorial 3   complete  0.94000      0.94000       
submission.csv  2019-05-08 11:49:42  Tutorial 3   complete  0.94250      0.94250       
submission.csv  2019-05-02 17:01:11  Tutorial 3   complete  0.93970      0.93970       
submission.csv  2019-04-25 18:36:36  Tutorial 3   complete  0.94020      0.94020       
submission.csv  2019-04-21 18:15:04  Tutorial 3   complete  0.93770      0.93770       
submission.csv  2019-04-21 11:00

In [33]:
fw.io.create_clean_dir(work_dir)